#Итоговый проект

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from os import listdir
import codecs
import pickle                                                                   
from bs4 import BeautifulSoup as BS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from boilerpipe.extract import Extractor
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/jpype/_core.py:209: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [2]:
train_data = pd.read_csv('train_groups.csv',dtype=np.int16)

In [3]:
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)       

In [ ]:
#START HEAR

In [4]:
name = 'NumWordsRulesExtractor'
name = 'train_data'+name

In [5]:
train_data = load_obj(name)

In [6]:
doc_to_title = {}
with open('docs_titles.tsv', encoding = 'UTF-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))


28026


In [7]:
doc_titles = []
for id in train_data.doc_id.values:
    doc_titles.append(doc_to_title[id])

In [8]:
train_data['title'] = doc_titles

In [9]:
def get_splits(data,n=3):
    bound =[]
    max_group = data.group_id.max()
    bound.extend([i*int(max_group/n)+1 for i in range(1,n)])
    bound.append(max_group+1)
    prev_id = 1
    for group_id in bound:
        yield (data[~((prev_id <= data.group_id) & (data.group_id < group_id))].pair_id.values-1,
            data[(prev_id <= data.group_id) & (data.group_id < group_id)].pair_id.values-1)
        prev_id = group_id

In [10]:
vectorizer = TfidfVectorizer()#max_features = 70)
#texts_vectorized = vectorizer.fit_transform(train_data.text)

In [11]:
#texts_vectorized.shape

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
groups_one_hot = OneHotEncoder().fit_transform(train_data.group_id.values.reshape(-1, 1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [14]:
groups_one_hot.shape

(11690, 129)

In [15]:
#texts_vectorized.shape

In [16]:
from scipy.sparse import hstack
from scipy.sparse import vstack

In [17]:
#features = hstack( [texts_vectorized,groups_one_hot] )

In [18]:
target = train_data.target.values

In [69]:
#doc_id = train_data.doc_id.values

In [19]:
groups = train_data.group_id.values

In [20]:
groups

array([  1,   1,   1, ..., 129, 129, 129], dtype=int16)

In [21]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.cluster import DBSCAN

In [36]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
for group in tqdm(set(groups)):
    elements_in_same_group = train_data[train_data.group_id == group].pair_id.values-1
    vectorizer.fit(train_data.title[elements_in_same_group] +train_data.text[elements_in_same_group])
    texts_vectorized_titles = vectorizer.transform(train_data.title[elements_in_same_group])
    texts_vectorized_titles = scaler.fit_transform(texts_vectorized_titles)
    
    texts_vectorized_texts = vectorizer.transform(train_data.text[elements_in_same_group])
    texts_vectorized_texts = scaler.fit_transform(texts_vectorized_texts)

    #dist_matr = pairwise_distances(texts_vectorized.tocsr()[elements_in_same_group], texts_vectorized.tocsr()[elements_in_same_group], metric = 'cosine')
    dist_matr = pairwise_distances(texts_vectorized_titles, texts_vectorized_titles, metric = 'cosine')
    dist_matr2 = pairwise_distances(texts_vectorized_texts, texts_vectorized_texts, metric = 'cosine')
    dist_matr3 = pairwise_distances(texts_vectorized_texts, texts_vectorized_titles, metric = 'cosine')
    #print(texts_vectorized.shape)
    #clustering = DBSCAN(eps = 0.5, min_samples=3).fit(dist_matr)
    dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:21][:,::-1]
    dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:21][:,::-1]
    dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:21][:,::-1]
    mean_dist = dist_matr.mean(axis = 1)
    dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist.reshape(-1,1)))
    dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted2))
    dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted3))

#    dist_matr_sorted = hstack((dist_matr_sorted, texts_vectorized))
    
    if group == 1 :
        features_by_dist =  dist_matr_sorted
    else:
        features_by_dist = vstack((features_by_dist, dist_matr_sorted))

In [37]:
features_by_dist.shape

(11690, 64)

In [38]:
features_by_dist = hstack([features_by_dist, groups_one_hot])

In [39]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
max_score = 0
scores = []
for train, test in get_splits(train_data,n=3):
            print(type(features_by_dist))
            model = RandomForestClassifier(n_jobs=6,n_estimators=200)
            model.fit(features_by_dist.tocsr()[train], target[train])
            pred = (model.predict(features_by_dist.tocsr()[test]))               
            scores.append(f1_score(pred,target[test]))
            print(scores)
print(sum(scores)/3)

<class 'scipy.sparse.coo.coo_matrix'>
[0.7205815538391639]
<class 'scipy.sparse.coo.coo_matrix'>
[0.7205815538391639, 0.690319310694374]
<class 'scipy.sparse.coo.coo_matrix'>
[0.7205815538391639, 0.690319310694374, 0.7282463186077645]
0.7130490610471009
